In [ ]:
from huggingface_hub import login
login(token = "")

In [ ]:
import os
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Set up YouTube API client
api_key = ''  # Replace with your YouTube Data API key
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
#Check for CUDA availability

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"  # You might need to adjust this based on Kaggle's available models
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [ ]:
# Create a pipeline for text generation
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)  # Use GPU

In [ ]:
def get_video_info(video_id):
    try:
        request = youtube.videos().list(part="snippet", id=video_id)
        response = request.execute()
        if 'items' in response and len(response['items']) > 0:
            return response['items'][0]['snippet']
        else:
            print(f"No video found with id: {video_id}")
            return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [ ]:
def get_transcript_with_timestamps(video_id):
    return YouTubeTranscriptApi.get_transcript(video_id)

In [ ]:
def format_timestamp(seconds):
    minutes, seconds = divmod(int(seconds), 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

In [ ]:
def get_relevant_segments(transcript, keywords):
    text_segments = [entry['text'] for entry in transcript]
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(text_segments + [' '.join(keywords)])
    
    cosine_similarities = (tfidf_matrix * tfidf_matrix[-1].T).A.flatten()[:-1]
    most_similar_indices = cosine_similarities.argsort()[-5:][::-1]  # Get top 5 relevant segments
    
    relevant_segments = [transcript[i] for i in most_similar_indices]
    return relevant_segments

In [ ]:
def get_dynamic_length(text_length, min_ratio=0.1, max_ratio=0.3, min_length=256, max_length=2048):
    """Calculate dynamic lengths for input and new tokens based on the input text length."""
    ratio = max(min_ratio, min(max_ratio, 1000 / text_length))
    dynamic_length = int(text_length * ratio)
    input_length = max(min_length, min(dynamic_length, max_length))
    new_tokens = max(64, min(input_length // 2, 512))  # Ensure at least 64 new tokens, at most 512
    return input_length, new_tokens

In [ ]:
def generate_bullet_points(transcript, keywords=None):
    if keywords:
        relevant_segments = get_relevant_segments(transcript, keywords)
    else:
        relevant_segments = transcript

    full_text = " ".join([segment['text'] for segment in relevant_segments])
    text_length = len(full_text.split())
    
    # Calculate dynamic lengths
    input_length, max_new_tokens = get_dynamic_length(text_length)
    
    prompt = f"Summarize the following text in 5 bullet points:\n\n{full_text}\n\nBullet points:"
    
    try:
        generated_text = text_generator(prompt, max_new_tokens=max_new_tokens, num_return_sequences=1)[0]['generated_text']
        bullet_points = generated_text.split("Bullet points:")[-1].strip().split('\n')
    except Exception as e:
        print(f"Error in bullet point generation: {str(e)}")
        bullet_points = sent_tokenize(full_text)[:5]

    # Assign timestamps to bullet points
    timestamped_bullets = []
    for i, bullet in enumerate(bullet_points):
        if i < len(relevant_segments):
            timestamp = format_timestamp(relevant_segments[i]['start'])
        else:
            timestamp = format_timestamp(relevant_segments[-1]['start'])
        timestamped_bullets.append(f"[{timestamp}] {bullet.strip('- ')}")

    return timestamped_bullets

In [ ]:
def chat_about_video(bullet_points, user_input, transcript_length):
    context = "Here are the main points of a video:\n" + "\n".join(bullet_points)
    prompt = f"{context}\n\nHuman: {user_input}\n\nAssistant:"
    
    # Calculate dynamic lengths for chat response
    input_length, max_new_tokens = get_dynamic_length(transcript_length, min_ratio=0.2, max_ratio=0.5, min_length=512, max_length=4096)
    
    response = text_generator(prompt, max_new_tokens=max_new_tokens, num_return_sequences=1)[0]['generated_text']
    return response.split("Assistant:")[-1].strip()

In [ ]:
def main():
    video_url = input("Enter the YouTube video URL: ")
    video_id = video_url.split("v=")[1]
    
    # Get video information
    video_info = get_video_info(video_id)
    if video_info:
        print(f"Title: {video_info['title']}")
        print(f"Description: {video_info['description'][:100]}...")
    
    try:
        # Get transcript with timestamps
        transcript = get_transcript_with_timestamps(video_id)
        transcript_length = sum(len(entry['text'].split()) for entry in transcript)
        print(f"Transcript length: {transcript_length} words")
        
        # Ask user for summary preference
        summary_type = input("Do you want a full summary or a summary based on specific keywords? (full/keywords): ").lower()
        
        if summary_type == 'keywords':
            keywords = input("Enter keywords (comma-separated): ").split(',')
            keywords = [k.strip() for k in keywords]
            bullet_points = generate_bullet_points(transcript, keywords)
        else:
            bullet_points = generate_bullet_points(transcript)
        
        print("\nVideo Summary (Bullet Points with Timestamps):")
        for bullet in bullet_points:
            print(bullet)
        
        # Chat loop
        print("\nYou can now chat about the video. Type 'quit' to exit.")
        while True:
            user_input = input("\nYou: ")
            if user_input.lower() == 'quit':
                break
            
            response = chat_about_video(bullet_points, user_input, transcript_length)
            print(f"Assistant: {response}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Unable to process the video. Please try another video or check your input.")

if __name__ == "__main__":
    main()